In [ ]:
# %% Deep learning - Section 19.177
#    Examine feature maps activations

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [1]:
# %% Libraries and modules
import numpy                  as np
import matplotlib.pyplot      as plt
import torch
import torch.nn               as nn
import seaborn                as sns
import copy
import torch.nn.functional    as F
import pandas                 as pd
import scipy.stats            as stats
import sklearn.metrics        as skm
import time
import sys
import imageio.v2             as imageio
import torchvision
import torchvision.transforms as T

from torch.utils.data                 import DataLoader,TensorDataset,Dataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from scipy.stats                      import zscore
from sklearn.decomposition            import PCA
from scipy.signal                     import convolve2d
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
plt.style.use('default')


In [2]:
# %% Generate data

# 2D Gaussian params
n_per_class = 1000
n_classes   = 2
img_size    = 91
x           = np.linspace(-4,4,img_size)
X,Y         = np.meshgrid(x,x)

widths      = [1.8,2.4]

# Preallocate tensors for images (N,channels,size,size) and labels (N)
images  = torch.zeros( n_classes*n_per_class,1,img_size,img_size )
labels  = torch.zeros( n_classes*n_per_class )

# Generate images
for i in range(n_classes*n_per_class):

    # Gaussian with random center offset (remainder trick for width, all even
    # images go into category 0, all odd images go into category 1)
    c = 2*np.random.randn(2)
    G = np.exp( -((X-c[0])**2 + (Y-c[1])**2 ) / (2*widths[i%2]**2) )

    # Layer some noise
    G = G + np.random.randn(img_size,img_size)/5

    # Add to tensor
    images[i,:,:,:] = torch.tensor(G).view(1,img_size,img_size)
    labels[i]       = i%2

labels = labels[:,None]


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5))/2
fig,axs = plt.subplots(3,7,figsize=(phi*7,7))

for i,ax in enumerate(axs.flatten()):

    pic = np.random.randint(2*n_per_class)
    G   = np.squeeze( images[pic,:,:] )

    ax.imshow(G,vmin=-1,vmax=1,cmap='jet')
    ax.set_title('Class %s'%int(labels[pic].item()))
    ax.set_xticks([])
    ax.set_yticks([])

plt.savefig('figure17_feature_maps.png')
plt.show()
files.download('figure17_feature_maps.png')


In [4]:
# %% Create train and test datasets

# Split data with scikitlearn (10% test data)
train_data,test_data,train_labels,test_labels = train_test_split(images,labels,test_size=0.1)

# Convert to PyTorch datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Convert into DataLoader objects
batch_size   = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])


In [6]:
# %% Function to generate the model

def gen_model():

    class mnist_CNN(nn.Module):
        def __init__(self):
            super().__init__()

            # Convolution layer 1
            # output size: (91+2*1-3)/1 + 1 = 91
            # post-pooling: 91/2 = 45
            self.conv1 = nn.Conv2d(1,6,kernel_size=3,stride=1,padding=1)

            # Convolution layer 2
            # output size: (45+2*1-3)/1 + 1 = 45
            # post-pooling: 45/2 = 22
            self.conv2 = nn.Conv2d(6,4,kernel_size=3,stride=1,padding=1)

            # Fully connected layer
            self.fc1 = nn.Linear(22*22*4,50)

            # Output layer
            self.output = nn.Linear(50,1)

        def forward(self,x):

            # Adapt to export
            # MaxPool and ReLu on convolution layer 1
            conv1_act = F.relu(self.conv1(x))
            x         = F.avg_pool2d(conv1_act,(2,2))

            # MaxPool and ReLu on convolution layer 2
            conv2_act = F.relu(self.conv2(x))
            x         = F.avg_pool2d(conv2_act,(2,2))

            # Vectorise for linear layer
            x = torch.flatten(x,start_dim=1)

            # Linear and output layers
            x = F.relu(self.fc1(x))
            x = self.output(x)

            return x,conv1_act,conv2_act

    # Create model instance
    CNN = mnist_CNN()

    # Loss function
    loss_fun = nn.BCEWithLogitsLoss()

    # Optimizer
    optimizer = torch.optim.Adam(CNN.parameters(),lr=0.001)

    return CNN,loss_fun,optimizer


In [ ]:
# %% Test the model on one batch

# test the model with one batch
CNN,loss_fun,optimizer = gen_model()

# test that the model runs and can compute a loss
X,y = next(iter(train_loader))
yHat,feat_map1,feat_map2 = CNN(X)
loss = loss_fun(yHat,y)

# check sizes of outputs
print('Predicted category:')
print(yHat.shape)
print('\nFeature map after conv1')
print(feat_map1.shape)
print('\nFeature map after conv2')
print(feat_map2.shape)


In [ ]:
# %% Check all the parameters in the model

summary(CNN,(1,img_size,img_size))


In [11]:
# %% Function to train the model

def train_model():

    # Parameters, model instance, inizialise vars
    num_epochs = 10
    CNN,loss_fun,optimizer = gen_model()

    train_losses = []
    test_losses  = []
    train_acc    = []
    test_acc     = []

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Loop over training batches
        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Forward propagation and loss (only select 1st output)
            yHat = CNN(X)[0]
            loss = loss_fun(yHat,y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Loss and accuracy from this batch
            batch_loss.append(loss.item())
            batch_acc.append( torch.mean( ((yHat>0)==y).float() ).item() )

        train_losses.append( np.mean(batch_loss) )
        train_acc.append( 100*np.mean(batch_acc) )

        # Test accuracy
        CNN.eval()

        with torch.no_grad():
            X,y = next(iter(test_loader))
            yHat = CNN(X)[0]
            loss = loss_fun(yHat,y)

            test_acc.append( 100*torch.mean( ((yHat>0)==y).float() ).item() )
            test_losses.append(loss.item())

        CNN.train()

    return train_acc,test_acc,train_losses,test_losses,CNN


In [129]:
# %% Run the model

# Takes ~30 secs
train_acc,test_acc,train_losses,test_losses,CNN = train_model()


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5)) / 2
fig,ax = plt.subplots(1,2,figsize=(1.5*phi*6,6))

ax[0].plot(train_losses,'s-',label='Train')
ax[0].plot(test_losses,'o-',label='Test')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss (MSE)')
ax[0].set_title('Model loss')

ax[1].plot(train_acc,'s-',label='Train')
ax[1].plot(test_acc,'o-',label='Test')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy (%)')
ax[1].set_title(f'Final model test accuracy: {test_acc[-1]:.2f}%')
ax[1].legend()

plt.savefig('figure18_feature_maps.png')
plt.show()
files.download('figure18_feature_maps.png')


In [ ]:
# %% Plotting

# Pass test data through model
X,y = next(iter(test_loader))
yHat,feat_map1,feat_map2 = CNN(X)

# Plot
phi = (1 + np.sqrt(5)) / 2
fig,axs = plt.subplots(2,10,figsize=(2*phi*5,5))

for i,ax in enumerate(axs.flatten()):

    G = torch.squeeze( X[i,0,:,:] ).detach()
    ax.imshow(G,vmin=-1,vmax=1,cmap='jet')
    t = ( int(y[i].item()), int(yHat[i].item()>0) )

    ax.set_title('T=%s, P=%s'%t)
    ax.set_xticks([])
    ax.set_yticks([])

plt.savefig('figure19_feature_maps.png')
plt.show()
files.download('figure19_feature_maps.png')


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5)) / 2
fig,axs = plt.subplots(7,10,figsize=(1.5*phi*6,6))

for pic_i in range(10):

    # Show the original picture
    img = X[pic_i,0,:,:].detach()
    axs[0,pic_i].imshow(img,cmap='jet',vmin=0,vmax=1)
    axs[0,pic_i].axis('off')
    axs[0,pic_i].text(2,2,'T=%s'%int(y[pic_i].item()),ha='left',va='top',color='w',fontweight='bold')

    for feat_i in range(6):

        # Extract the feature map from this image
        img = feat_map1[pic_i,feat_i,:,:].detach()
        axs[feat_i+1,pic_i].imshow(img,cmap='plasma',vmin=0,vmax=torch.max(img)*.9)
        axs[feat_i+1,pic_i].axis('off')
        axs[feat_i+1,pic_i].text(-5,45,feat_i,ha='right') if pic_i==0 else None

plt.tight_layout()
plt.suptitle('First set of feature map activations for 10 test images',x=.5,y=1.01)

plt.savefig('figure20_feature_maps.png')
plt.show()
files.download('figure20_feature_maps.png')


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5)) / 2
fig,axs = plt.subplots(5,10,figsize=(1.5*phi*6,6))

for pic_i in range(10):

    # Show the original picture
    img = X[pic_i,0,:,:].detach()
    axs[0,pic_i].imshow(img,cmap='jet',vmin=0,vmax=1)
    axs[0,pic_i].axis('off')
    axs[0,pic_i].text(2,2,'T=%s'%int(y[pic_i].item()),ha='left',va='top',color='w',fontweight='bold')

    for feat_i in range(4):

        # Extract the feature map from this image
        img = feat_map2[pic_i,feat_i,:,:].detach()
        axs[feat_i+1,pic_i].imshow(img,cmap='plasma',vmin=0,vmax=torch.max(img)*.9)
        axs[feat_i+1,pic_i].axis('off')
        axs[feat_i+1,pic_i].text(-5,22,feat_i,ha='right') if pic_i==0 else None

plt.tight_layout()
plt.suptitle('Second set of feature map activations for 10 test images',x=.5,y=1.01)

plt.savefig('figure21_feature_maps.png')
plt.show()
files.download('figure21_feature_maps.png')


In [136]:
# %% Spatial correlations across feature maps in convolutional layer 1

# Variables
n_stim  = feat_map1.shape[0]
n_maps  = feat_map1.shape[1]
n_corrs = (n_maps*(n_maps-1))//2

all_rs    = np.zeros((n_stim,n_corrs))
all_corrs = np.zeros((n_maps,n_maps))

# Loop over each stimulus (input images)
for i in range(n_stim):

    # Vectorised feature maps from current image
    feat_maps = feat_map1[i,:,:,:].view(n_maps,-1).detach()

    # Correlation matrix
    C = np.corrcoef(feat_maps)
    all_corrs += C

    # extract the unique correlations from the matrix
    idx = np.nonzero(np.triu(C,1))
    all_rs[i,:] = C[idx]


In [ ]:
# %% Plotting

# define the x-axis labels
xlab = []*n_corrs
for i in range(n_corrs):
    xlab.append('%s-%s' %(idx[0][i],idx[1][i]))

# Plot
phi = (1 + np.sqrt(5)) / 2
fig,(ax0,ax1) = plt.subplots(1,2,figsize=(2*phi*5,5))

cmap = plt.cm.plasma(np.linspace(0.2,0.9,n_corrs))

for i in range(n_corrs):
    ax0.plot(i+np.random.randn(n_stim)/30,all_rs[:,i],'o',color=cmap[i],markerfacecolor='w',markersize=8)

ax0.set_xlim([-.5,n_corrs-.5])
ax0.set_ylim([-1.05,1.05])
ax0.set_xticks(range(n_corrs))
ax0.set_xticklabels(xlab)
ax0.set_xlabel('Feature map pair')
ax0.set_ylabel('Correlation coefficient')
ax0.set_title('Correlations for each image\n(Conv. layer 1)')

# Average correlation matrix
h = ax1.imshow(all_corrs/n_stim,vmin=-1,vmax=1,cmap='plasma')
ax1.set_title('Correlation matrix')
ax1.set_xlabel('Feature map')
ax1.set_ylabel('Feature map')
fig.colorbar(h,ax=ax1)
plt.tight_layout()

plt.savefig('figure22_feature_maps.png')
plt.show()
files.download('figure22_feature_maps.png')


In [138]:
# %% Spatial correlations across feature maps in convolutional layer 2

# Variables
n_stim  = feat_map2.shape[0]
n_maps  = feat_map2.shape[1]
n_corrs = (n_maps*(n_maps-1))//2

all_rs    = np.zeros((n_stim,n_corrs))
all_corrs = np.zeros((n_maps,n_maps))

# Loop over each stimulus (input images)
for i in range(n_stim):

    # Vectorised feature maps from current image
    feat_maps = feat_map2[i,:,:,:].view(n_maps,-1).detach()

    # Correlation matrix
    C = np.corrcoef(feat_maps)
    all_corrs += C

    # extract the unique correlations from the matrix
    idx = np.nonzero(np.triu(C,1))
    all_rs[i,:] = C[idx]


In [ ]:
# %% Plotting

# define the x-axis labels
xlab = []*n_corrs
for i in range(n_corrs):
    xlab.append('%s-%s' %(idx[0][i],idx[1][i]))

# Plot
phi = (1 + np.sqrt(5)) / 2
fig,(ax0,ax1) = plt.subplots(1,2,figsize=(2*phi*5,5))

cmap = plt.cm.plasma(np.linspace(0.2,0.9,n_corrs))

for i in range(n_corrs):
    ax0.plot(i+np.random.randn(n_stim)/30,all_rs[:,i],'o',color=cmap[i],markerfacecolor='w',markersize=8)

ax0.set_xlim([-.5,n_corrs-.5])
ax0.set_ylim([-1.05,1.05])
ax0.set_xticks(range(n_corrs))
ax0.set_xticklabels(xlab)
ax0.set_xlabel('Feature map pair')
ax0.set_ylabel('Correlation coefficient')
ax0.set_title('Correlations for each image\n(Conv. layer 2)')

# Average correlation matrix
h = ax1.imshow(all_corrs/n_stim,vmin=-1,vmax=1,cmap='plasma')
ax1.set_title('Correlation matrix')
ax1.set_xlabel('Feature map')
ax1.set_ylabel('Feature map')
fig.colorbar(h,ax=ax1)
plt.tight_layout()

plt.savefig('figure23_feature_maps.png')
plt.show()
files.download('figure23_feature_maps.png')


In [106]:
# %% Exercise 1
#    The code here grabs the activation of the feature maps *after* relu is applied. Modify the code to get the *pre-relu*
#    activations, which corresponds to the linear part of the activations. Redraw the maps (do you need to modify the
#    color range?). Do they look similar or different before vs after relu? Which maps are more relevant to interpret
#    when trying to understand how the CNN works and how it represents data?

# Function to generate the model
def gen_model():

    class mnist_CNN(nn.Module):
        def __init__(self):
            super().__init__()

            # Convolution layer 1
            # output size: (91+2*1-3)/1 + 1 = 91
            # post-pooling: 91/2 = 45
            self.conv1 = nn.Conv2d(1,6,kernel_size=3,stride=1,padding=1)

            # Convolution layer 2
            # output size: (45+2*1-3)/1 + 1 = 45
            # post-pooling: 45/2 = 22
            self.conv2 = nn.Conv2d(6,4,kernel_size=3,stride=1,padding=1)

            # Fully connected layer
            self.fc1 = nn.Linear(22*22*4,50)

            # Output layer
            self.output = nn.Linear(50,1)

        def forward(self,x):

            # Adapt to export
            # MaxPool and ReLu on convolution layer 1
            conv1_act_pre = self.conv1(x)
            conv1_act     = F.relu(conv1_act_pre)
            x             = F.avg_pool2d(conv1_act,(2,2))

            # MaxPool and ReLu on convolution layer 2
            conv2_act_pre = self.conv2(x)
            conv2_act     = F.relu(conv2_act_pre)
            x             = F.avg_pool2d(conv2_act,(2,2))

            # Vectorise for linear layer
            x = torch.flatten(x,start_dim=1)

            # Linear and output layers
            x = F.relu(self.fc1(x))
            x = self.output(x)

            return x,conv1_act_pre,conv2_act_pre

    # Create model instance
    CNN = mnist_CNN()

    # Loss function
    loss_fun = nn.BCEWithLogitsLoss()

    # Optimizer
    optimizer = torch.optim.Adam(CNN.parameters(),lr=0.001)

    return CNN,loss_fun,optimizer


In [128]:
# %% Exercise 2
#    Relatedly, how much difference would it make to export the activations after the pooling layers? Which is better
#    and what are the advantages of inspecting the pre- vs. post-pooled activation maps?

# Function to generate the model
def gen_model():

    class mnist_CNN(nn.Module):
        def __init__(self):
            super().__init__()

            # Convolution layer 1
            # output size: (91+2*1-3)/1 + 1 = 91
            # post-pooling: 91/2 = 45
            self.conv1 = nn.Conv2d(1,6,kernel_size=3,stride=1,padding=1)

            # Convolution layer 2
            # output size: (45+2*1-3)/1 + 1 = 45
            # post-pooling: 45/2 = 22
            self.conv2 = nn.Conv2d(6,4,kernel_size=3,stride=1,padding=1)

            # Fully connected layer
            self.fc1 = nn.Linear(22*22*4,50)

            # Output layer
            self.output = nn.Linear(50,1)

        def forward(self,x):

            # Adapt to export
            # MaxPool and ReLu on convolution layer 1
            conv1_act_pre  = self.conv1(x)
            conv1_act      = F.relu(conv1_act_pre)
            conv1_act_post = F.avg_pool2d(conv1_act,(2,2))

            # MaxPool and ReLu on convolution layer 2
            conv2_act_pre  = self.conv2(conv1_act_post) # Corrected: Pass conv1_act_post here
            conv2_act      = F.relu(conv2_act_pre)
            conv2_act_post = F.avg_pool2d(conv2_act,(2,2))

            # Vectorise for linear layer
            x = torch.flatten(conv2_act_post,start_dim=1)

            # Linear and output layers
            x = F.relu(self.fc1(x))
            x = self.output(x)

            return x,conv1_act_post,conv2_act_post

    # Create model instance
    CNN = mnist_CNN()

    # Loss function
    loss_fun = nn.BCEWithLogitsLoss()

    # Optimizer
    optimizer = torch.optim.Adam(CNN.parameters(),lr=0.001)

    return CNN,loss_fun,optimizer
